# 医療テキスト分析システム

このノートブックでは以下の機能を実行できます：
1. ダミーデータの生成
2. LLMサーバーの起動確認
3. テキスト分析の実行
4. 結果の確認と保存

In [113]:
import sys
import os
import pandas as pd
import requests
from IPython.display import display
from openai import OpenAI

# srcディレクトリをパスに追加
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.data.data_generator import MedicalDataGenerator
from src.analyzer.excel_analyzer import ExcelAnalyzer

## 1. LLMサーバーの起動確認

**注意**: 別ターミナルで `python src/analyzer/llm_server.py` を実行してサーバーを起動しておく必要があります。

In [114]:
def check_llm_server():
    server_url = "http://localhost:8000"
    try:
        # ヘルスチェックエンドポイントを確認
        health_response = requests.get(f"{server_url}/health", timeout=5)
        if health_response.status_code == 200:
            print("✅ LLMサーバーが正常に動作しています")
            
            # APIエンドポイントも確認
            api_response = requests.get(f"{server_url}/v1/chat/completions", timeout=5)
            if api_response.status_code == 405:  # POSTメソッドのみ許可される場合
                print("✅ APIエンドポイントも利用可能です")
                return True
            else:
                print("⚠️ APIエンドポイントが応答しませんでした")
                print(f"ステータスコード: {api_response.status_code}")
                return False
    except requests.exceptions.ConnectionError:
        print("❌ LLMサーバーに接続できません")
        print(f"サーバーURL: {server_url}")
        print("\n以下を確認してください：")
        print("1. サーバーが起動しているか")
        print("2. URLが正しいか")
        print("3. ファイアウォールの設定")
        print("\nサーバー起動コマンド：")
        print("python src/analyzer/llm_server.py")
    except requests.exceptions.Timeout:
        print("❌ サーバーの応答がタイムアウトしました")
    except Exception as e:
        print(f"❌ 予期せぬエラーが発生しました: {str(e)}")
    return False

# サーバーの状態を確認
check_llm_server()

✅ LLMサーバーが正常に動作しています
✅ APIエンドポイントも利用可能です


True

### テスト呼び出し

In [115]:
# OpenAIクライアントの初期化
client = OpenAI(
    api_key="EMPTY",
    base_url="http://localhost:8000/v1"
)

try:
    # テスト呼び出し
    completion = client.completions.create(
        model="Qwen/Qwen2.5-72B-Instruct-GPTQ-Int4",
        prompt="San Francisco is a"
    )
    print("✅ LLMサーバーのテスト呼び出しに成功しました")
    print("応答結果:", completion)
except Exception as e:
    print("❌ LLMサーバーのテスト呼び出しに失敗しました")
    print(f"エラー内容: {str(e)}")


✅ LLMサーバーのテスト呼び出しに成功しました
応答結果: Completion(id='cmpl-26765425ea9f4c90ac4ec8ad3f7110cd', choices=[CompletionChoice(finish_reason='length', index=0, logprobs=None, text=' city where neighbors still know each other and look out for one another.\nThe City', stop_reason=None, prompt_logprobs=None)], created=1739420159, model='Qwen/Qwen2.5-72B-Instruct-GPTQ-Int4', object='text_completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=16, prompt_tokens=4, total_tokens=20, completion_tokens_details=None, prompt_tokens_details=None))


## 2. テキスト分析の実行

生成したデータに対して、各種分析を実行します。

In [116]:
from analyzer.excel_analyzer import ExcelAnalyzer

def main():
    # ExcelAnalyzerのインスタンス化
    analyzer = ExcelAnalyzer(
        file_path="../data/sample_data.xlsx",
        llm_server_url="http://localhost:8000",
        template_path="../templates/prompt_templates.json"
    )

    # Excelファイルの読み込み
    if not analyzer.load_excel():
        print("Excelファイルの読み込みに失敗しました")
        return

    # データの基本情報を表示
    analyzer.display_data_info()

    # テンプレートを使用して順番に分析を実行
    analysis_templates = [
        "cancer_diagnosis",    # がん診断名
        "cancer_stage",        # がんステージ
        "diagnostic_test",     # 診断検査
        "first_treatment",     # 初回治療
        "chemotherapy_info",   # 化学療法情報
        "surgery_type",        # 手術術式
        "special_notes"        # 特記事項
    ]

    for template_key in analysis_templates:
        print(f"\n{template_key}の分析を開始します...")
        if not analyzer.analyze_with_template(template_key):
            print(f"{template_key}の分析に失敗しました")

    # 分析結果を保存
    analyzer.save_results("analyzed_results.xlsx")

if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'analyzer'

In [117]:
# アナライザーの初期化
try:
    analyzer = ExcelAnalyzer(
        file_path="../data/sample_data.xlsx",
        llm_server_url="http://localhost:8000",
        template_path="../templates/prompt_templates.json"
    )
    
    # ファイルの読み込み
    if not analyzer.load_excel():
        print("❌ ファイルの読み込みに失敗しました")
    else:
        print("✅ ファイルの読み込みが完了しました")
        analyzer.display_data_info()
except Exception as e:
    print(f"❌ エラーが発生しました: {str(e)}")
    print("以下を確認してください：")
    print("1. ../templates/prompt_templates.json ファイルが存在すること")
    print("2. ../data/sample_data.xlsx ファイルが存在すること")
    print("3. LLMサーバーが起動していること")

# 各種分析の実行
analysis_templates = [
    "cancer_diagnosis",   # がん診断名の抽出
    #"cancer_stage",       # ステージ情報の抽出
    #"diagnostic_test",    # 診断検査の抽出
    #"first_treatment",    # 初回治療情報の抽出
    #"chemotherapy_info",  # 抗がん剤治療情報の抽出
    #"surgery_type",       # 術式の抽出
    #"special_notes"       # 特記事項の抽出
]

# 分析の実行
for template in analysis_templates:
    print(f"\n分析実行中: {template}")
    if not analyzer.analyze_with_template(template):
        print(f"警告: {template}の分析に失敗しました")

# 結果の保存
analyzer.save_results()

テンプレートを読み込みました（7件）
ファイルの読み込みが完了しました
✅ ファイルの読み込みが完了しました
データ行数: 6

列一覧:
- ID
- day
- text

分析実行中: cancer_diagnosis
警告: ID 1 の分析中にエラーが発生: API呼び出し中にエラーが発生: APIエラー: 400 - {"object":"error","message":"[{'type': 'missing', 'loc': ('body', 'model'), 'msg': 'Field required', 'input': {'messages': [{'role': 'system', 'content': '与えられた医療テキストからがんの診断名を抽出してください。\\n\\n抽出ルール:\\n- がんの正式な診断名を抽出\\n- 例：卵巣がん、子宮体癌、肺腺癌など\\n- 組織型の情報があれば含める\\n- 複数の診断がある場合は最新のものを採用\\n- 明確な診断名がない場合は \\'N/A\\' を返す\\n\\n有効な出力例:\\n- \"漿液性卵巣癌\"\\n- \"子宮体部類内膜癌\"\\n- \"N/A\"\\n\\n無効な出力例:\\n- \"がんの疑い\"\\n- \"悪性腫瘍\"'}, {'role': 'user', 'content': '以下のテキストについて質問に答えてください。\\n\\nテキスト: [2023-01-01]\\n組織生検の結果、子宮内膜癌、Stage ICと診断。 心機能低下（EF 49%）あり\\n\\n[2023-01-22]\\n治療方針：開腹子宮全摘出術の方針。 心機能低下（EF 53%）あり\\n\\n[2023-02-08]\\n手術記録：腹腔鏡下全摘術施行。手術時間125分、出血量271ml。\\n\\n質問: がん診断名抽出'}], 'temperature': 0.7, 'max_tokens': 512}}]","type":"BadRequestError","param":null,"code":400}
警告: ID 2 の分析中にエラーが発生: API呼び出し中にエラーが発生: APIエラー: 400 - {"object":"error","message":"[

True

## 3. 分析結果の確認と保存

In [62]:
# 分析結果の保存
output_path = "../data/sample_data_analyzed.xlsx"
analyzer.save_results(output_path)

# 結果の確認
results_df = pd.read_excel(output_path)
print("\n分析結果のプレビュー:")
display(results_df.head())

分析結果を '../data/sample_data_analyzed.xlsx' に保存しました

分析結果の概要:
- 分析結果_がん診断名抽出:
  総データ数: 31
  ユニークな値の数: 1
  未検出(N/A)の数: 31
  主な抽出結果:
    - N/A: 31件
- 分析結果_がんステージ抽出:
  総データ数: 31
  ユニークな値の数: 1
  未検出(N/A)の数: 31
  主な抽出結果:
    - N/A: 31件
- 分析結果_診断検査抽出:
  総データ数: 31
  ユニークな値の数: 1
  未検出(N/A)の数: 31
  主な抽出結果:
    - N/A: 31件
- 分析結果_初回治療情報抽出:
  総データ数: 31
  ユニークな値の数: 1
  未検出(N/A)の数: 31
  主な抽出結果:
    - N/A: 31件
- 分析結果_抗がん剤治療情報抽出:
  総データ数: 31
  ユニークな値の数: 1
  未検出(N/A)の数: 31
  主な抽出結果:
    - N/A: 31件
- 分析結果_術式抽出:
  総データ数: 31
  ユニークな値の数: 1
  未検出(N/A)の数: 31
  主な抽出結果:
    - N/A: 31件
- 分析結果_特記事項抽出:
  総データ数: 31
  ユニークな値の数: 1
  未検出(N/A)の数: 31
  主な抽出結果:
    - N/A: 31件

分析結果のプレビュー:


,ID,day,text,分析結果_がん診断名抽出,分析結果_がんステージ抽出,分析結果_診断検査抽出,分析結果_初回治療情報抽出,分析結果_抗がん剤治療情報抽出,分析結果_術式抽出,分析結果_特記事項抽出
0,1,2023-01-01,組織診の結果、明細胞癌、Stage IIAと診断。,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2023-01-15,治療方針：腹腔鏡下全摘術の方針。,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2023-02-13,手術記録：腹腔鏡下子宮摘出術施行。手術時間156分、出血量663ml。,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2023-03-02,術後化学療法としてDTX+CBDCAを開始。,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,2023-03-20,術後化学療法としてドセタキセル+カルボプラチンを開始。 糖尿病（HbA1c 9.6%）あり,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 4. 特定の患者の詳細確認

In [63]:
def display_patient_details(df, patient_id):
    patient_data = df[df['ID'] == patient_id].sort_values('day')
    print(f"患者ID {patient_id} の経過:")
    
    for _, row in patient_data.iterrows():
        print(f"\n日付: {row['day']}")
        print(f"記録: {row['text']}")
        
        # 分析結果列の表示
        analysis_columns = [col for col in df.columns if col.startswith('分析結果_')]
        for col in analysis_columns:
            if row[col] != 'N/A':
                print(f"{col}: {row[col]}")

# 最初の患者IDについて詳細を表示
first_patient_id = results_df['ID'].iloc[0]
display_patient_details(results_df, first_patient_id)

患者ID 1 の経過:

日付: 2023-01-01 00:00:00
記録: 組織診の結果、明細胞癌、Stage IIAと診断。
分析結果_がん診断名抽出: nan
分析結果_がんステージ抽出: nan
分析結果_診断検査抽出: nan
分析結果_初回治療情報抽出: nan
分析結果_抗がん剤治療情報抽出: nan
分析結果_術式抽出: nan
分析結果_特記事項抽出: nan

日付: 2023-01-15 00:00:00
記録: 治療方針：腹腔鏡下全摘術の方針。
分析結果_がん診断名抽出: nan
分析結果_がんステージ抽出: nan
分析結果_診断検査抽出: nan
分析結果_初回治療情報抽出: nan
分析結果_抗がん剤治療情報抽出: nan
分析結果_術式抽出: nan
分析結果_特記事項抽出: nan

日付: 2023-02-13 00:00:00
記録: 手術記録：腹腔鏡下子宮摘出術施行。手術時間156分、出血量663ml。
分析結果_がん診断名抽出: nan
分析結果_がんステージ抽出: nan
分析結果_診断検査抽出: nan
分析結果_初回治療情報抽出: nan
分析結果_抗がん剤治療情報抽出: nan
分析結果_術式抽出: nan
分析結果_特記事項抽出: nan

日付: 2023-03-02 00:00:00
記録: 術後化学療法としてDTX+CBDCAを開始。
分析結果_がん診断名抽出: nan
分析結果_がんステージ抽出: nan
分析結果_診断検査抽出: nan
分析結果_初回治療情報抽出: nan
分析結果_抗がん剤治療情報抽出: nan
分析結果_術式抽出: nan
分析結果_特記事項抽出: nan

日付: 2023-03-20 00:00:00
記録: 術後化学療法としてドセタキセル+カルボプラチンを開始。 糖尿病（HbA1c 9.6%）あり
分析結果_がん診断名抽出: nan
分析結果_がんステージ抽出: nan
分析結果_診断検査抽出: nan
分析結果_初回治療情報抽出: nan
分析結果_抗がん剤治療情報抽出: nan
分析結果_術式抽出: nan
分析結果_特記事項抽出: nan

日付: 2023-03-30 00:00:00
記録: 術後化学療法としてTCを開始。
分析結果_がん診断名抽出: nan
分析